🎥 Recommended Video: [What are GANs](https://www.youtube.com/watch?v=TpMIssRdhco)

🎥 Recommended Video: [What are Autoencoders](https://www.youtube.com/watch?v=qiUEgSCyY5o)

🎥 Recommended Video: [Diffusion models explained in 4-difficulty levels](https://www.youtube.com/watch?v=yTAMrHVG1ew)

## **8. Generative Models**

### **8.1 What are Generative Models?**
Generative models are a class of machine learning models designed to generate new data samples that resemble a given dataset. They learn the underlying distribution of the data and can create realistic images, text, audio, and more.

#### **Key Applications of Generative Models**:
- **Image Synthesis**: Creating realistic images (e.g., faces, landscapes).
- **Data Augmentation**: Generating additional training data.
- **Style Transfer**: Transforming images into different artistic styles.
- **Super-Resolution**: Enhancing the resolution of images.

---

### **8.2 Types of Generative Models**
There are several types of generative models, each with its own strengths and use cases:

#### **8.2.1 Generative Adversarial Networks (GANs)**
- Consist of two neural networks: a **generator** and a **discriminator**.
- The generator creates fake data, while the discriminator tries to distinguish between real and fake data.
- They are trained simultaneously in a competitive manner.

#### **8.2.2 Variational Autoencoders (VAEs)**
- Encode input data into a latent space and decode it back to the original data.
- Focus on learning a probabilistic representation of the data.
- Useful for generating diverse and smooth interpolations between data points.

#### **8.2.3 Diffusion Models**
- Gradually add noise to data and then learn to reverse the process to generate new samples.
- Known for producing high-quality images.

---

### **8.3 Code Example: Building a GAN**
Let’s build a simple GAN using TensorFlow/Keras to generate handwritten digits (MNIST dataset).

```python
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Define the generator
def build_generator(latent_dim):
    model = models.Sequential([
        layers.Dense(128, input_dim=latent_dim, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(28 * 28, activation='tanh'),  # Output layer
        layers.Reshape((28, 28, 1))
    ])
    return model

# Define the discriminator
def build_discriminator():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Output layer
    ])
    return model

# Build and compile the GAN
latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator()

discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Combined GAN model
discriminator.trainable = False
gan_input = layers.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = models.Model(gan_input, gan_output)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Load the MNIST dataset
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 127.5 - 1  # Normalize to [-1, 1]
x_train = np.expand_dims(x_train, axis=-1)

# Training loop
batch_size = 64
epochs = 10000

for epoch in range(epochs):
    # Train the discriminator
    real_images = x_train[np.random.randint(0, x_train.shape[0], batch_size)]
    real_labels = np.ones((batch_size, 1))

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_images = generator.predict(noise)
    fake_labels = np.zeros((batch_size, 1))

    d_loss_real = discriminator.train_on_batch(real_images, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print progress
    if epoch % 1000 == 0:
        print(f"Epoch: {epoch}, D Loss: {d_loss[0]}, G Loss: {g_loss}")

# Generate and display samples
noise = np.random.normal(0, 1, (16, latent_dim))
generated_images = generator.predict(noise)

plt.figure(figsize=(4, 4))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(generated_images[i, :, :, 0], cmap='gray')
    plt.axis('off')
plt.show()
```

#### **Explanation**:
1. The generator creates fake images from random noise.
2. The discriminator distinguishes between real and fake images.
3. The GAN is trained in a competitive manner, with the generator improving over time.

---

### **8.4 Code Example: Building a VAE**
Let’s build a Variational Autoencoder (VAE) using TensorFlow/Keras to generate new images.

```python
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

# Define the encoder
def build_encoder(latent_dim):
    inputs = layers.Input(shape=(28, 28, 1))
    x = layers.Flatten()(inputs)
    x = layers.Dense(256, activation='relu')(x)
    z_mean = layers.Dense(latent_dim)(x)
    z_log_var = layers.Dense(latent_dim)(x)
    return models.Model(inputs, [z_mean, z_log_var])

# Define the decoder
def build_decoder(latent_dim):
    inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(256, activation='relu')(inputs)
    x = layers.Dense(28 * 28, activation='sigmoid')(x)
    outputs = layers.Reshape((28, 28, 1))(x)
    return models.Model(inputs, outputs)

# Define the VAE
latent_dim = 2
encoder = build_encoder(latent_dim)
decoder = build_decoder(latent_dim)

inputs = layers.Input(shape=(28, 28, 1))
z_mean, z_log_var = encoder(inputs)

# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.random.normal(shape=tf.shape(z_mean))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])
outputs = decoder(z)
vae = models.Model(inputs, outputs)

# Define the VAE loss
def vae_loss(inputs, outputs):
    reconstruction_loss = tf.reduce_mean(tf.square(inputs - outputs))
    kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
    return reconstruction_loss + kl_loss

vae.compile(optimizer='adam', loss=vae_loss)

# Load the MNIST dataset
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train / 255.0  # Normalize to [0, 1]
x_train = np.expand_dims(x_train, axis=-1)

# Train the VAE
vae.fit(x_train, x_train, epochs=10, batch_size=128)

# Generate and display samples
n = 15
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))

grid_x = np.linspace(-2, 2, n)
grid_y = np.linspace(-2, 2, n)

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = decoder.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='gray')
plt.axis('off')
plt.show()
```

#### **Explanation**:
1. The encoder maps input images to a latent space.
2. The decoder generates images from the latent space.
3. The VAE is trained to minimize both reconstruction loss and KL divergence.

---

### **8.5 Code Example: Diffusion Models**
Let’s use a pre-trained diffusion model from Hugging Face to generate images.

```python
from diffusers import DDPMPipeline
import matplotlib.pyplot as plt

# Load a pre-trained diffusion model
pipeline = DDPMPipeline.from_pretrained('google/ddpm-cifar10-32')

# Generate an image
image = pipeline().images[0]

# Display the image
plt.imshow(image)
plt.axis('off')
plt.show()
```

#### **Explanation**:
1. The diffusion model gradually adds noise to data and then reverses the process to generate new samples.
2. The pre-trained model generates high-quality images.

---

### **8.6 In Conclusion**
- Generative models create new data samples that resemble a given dataset.
- GANs, VAEs, and diffusion models are popular types of generative models.
- These models have applications in image synthesis, data augmentation, and more.